In [1]:
!pip install requests confluent_kafka --quiet


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.8/3.8 MB 28.4 MB/s eta 0:00:00


In [2]:
!pip install langdetect


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 981.5/981.5 kB 6.7 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for langdetect: filename=langdetect-1.0.9-py3-none-any.whl size=993223 sha256=940f78e5dfaf51eb25798babdb6cf62c3f655a5cf77392256a3afc11b1fe94e7
  Stored in directory: /root/.cache/pip/wheels/0a/f2/b2/e5ca405801e05eb7c8ed5b3b4bcf1fcabcd6272c167640072e
Successfully built langdetect


In [3]:
import requests
import time
import pandas as pd
from confluent_kafka import Producer
import json
import re
from langdetect import detect

In [4]:
# get api keys
from google.colab import userdata

RAPIDAPI_KEY = userdata.get("RAPIDAPI_KEY")
CONFLUENT_API_KEY = userdata.get("CONFLUENT_API_KEY")
CONFLUENT_API_SECRET = userdata.get("CONFLUENT_API_SECRET")
BOOTSTRAP_SERVER = userdata.get("BOOTSTRAP_SERVER")


In [5]:
#connect to confluent cloud
from confluent_kafka import Producer

conf = {
    'bootstrap.servers': BOOTSTRAP_SERVER,
    'security.protocol': 'SASL_SSL',
    'sasl.mechanism': 'PLAIN',
    'sasl.username': CONFLUENT_API_KEY,
    'sasl.password': CONFLUENT_API_SECRET
}

producer = Producer(**conf)


In [6]:
import os

headers = {
    "x-rapidapi-key": RAPIDAPI_KEY,
    "x-rapidapi-host": "linkedin-data-api.p.rapidapi.com"
}


## Why we use skill_map
The skill_map is used only during preprocessing to extract a structured list of skills from unstructured job descriptions, since the LinkedIn API does not always return explicit skill fields. It helps populate a consistent skills_required column for each job. However, the actual recommendation logic is based on sentence embeddings and cosine similarity — not on this mapping. So skill_map is just a lightweight feature engineering step, not a rule-based recommender.

In [7]:
#feature engineering
skill_map = {
    "data": [
        "python", "sql", "tableau", "powerbi", "excel", "pandas", "numpy", "data wrangling",
        "data modeling", "statistics", "regression", "classification", "jupyter", "seaborn",
        "matplotlib", "scikit-learn", "exploratory data analysis", "data visualization",
        "machine learning", "mlflow", "feature engineering", "dash", "plotly", "lookml"
    ],
    "analyst": [
        "business analysis", "data analysis", "pivot tables", "vlookup", "excel", "storytelling",
        "requirement gathering", "gap analysis", "dashboarding", "root cause analysis", "kpis",
        "competitive analysis", "financial modeling", "forecasting", "google sheets", "sql"
    ],
    "scientist": [
        "deep learning", "pytorch", "keras", "tensorflow", "nlp", "transformers", "xgboost",
        "lightgbm", "catboost", "model tuning", "shap", "lime", "hyperparameter tuning",
        "model interpretability", "feature selection", "unsupervised learning"
    ],
    "engineer": [
        "python", "java", "c++", "aws", "azure", "gcp", "spark", "hadoop", "kafka", "flink",
        "airflow", "data pipelines", "bigquery", "snowflake", "dbt", "databricks", "linux",
        "bash", "etl", "nosql", "mongodb", "postgresql", "mysql", "terraform", "docker", "kubernetes"
    ],
    "developer": [
        "javascript", "typescript", "react", "vue", "angular", "nodejs", "flask", "django",
        "express", "graphql", "rest api", "html", "css", "sass", "nextjs", "vite", "webpack",
        "jest", "cypress", "redux", "storybook", "unit testing", "full stack"
    ],
    "product": [
        "product roadmap", "user stories", "agile", "scrum", "kanban", "mvp", "figma",
        "wireframes", "prototyping", "stakeholder management", "a/b testing", "market research",
        "competitive analysis", "user research", "confluence", "jira", "design thinking"
    ],
    "marketing": [
        "seo", "sem", "content marketing", "social media", "campaign management", "copywriting",
        "email marketing", "google analytics", "meta ads", "hubspot", "salesforce marketing cloud",
        "adwords", "customer segmentation", "crm", "brand positioning", "growth marketing"
    ],
    "manager": [
        "people management", "project management", "resource planning", "scrum", "agile",
        "leadership", "conflict resolution", "kpi tracking", "roadmap planning", "okrs",
        "jira", "performance reviews", "stakeholder communication", "risk management"
    ],
    "cloud": [
        "aws", "gcp", "azure", "iam", "vpc", "s3", "rds", "ec2", "cloudwatch", "lambda", "gke",
        "eks", "cloudformation", "terraform", "containerization", "devops", "cloud security",
        "serverless", "load balancer", "cloud architecture", "dns", "cdn"
    ]
}


keywords = ["data", "analyst", "scientist","developer", "engineer", "product", "manager","cloud"]
seen_jobs = set()


## What the helper functions do
These functions are used to enrich each job posting with structured metadata for later recommendation and filtering.

extract_skills(title, description) uses a predefined skill map to infer relevant skills from the job’s title and description.

extract_job_level(title) classifies the job as “Junior,” “Mid,” or “Senior” based on keywords in the title.

estimate_salary(title) provides an estimated salary range for the job using title-based heuristics (e.g., “Senior” or “Intern”), helping simulate real-world compensation info when not provided by the API.
These features are useful for downstream steps like skill gap analysis, vector embedding, and personalized recommendations.

In [8]:
#Function to extract relevant skills from job description

def extract_skills(title, description=""):
    title_words = title.lower().split()
    matched_categories = []

    # Match skill_map keys based on words in title
    for category in skill_map.keys():
        if any(word in category for word in title_words):
            matched_categories.append(category)

    # Collect skills from matched categories
    matched_skills = []
    for category in matched_categories:
        matched_skills.extend(skill_map[category])

    # Add any skills found directly in description
    full_text = (title + " " + description).lower()
    for skills in skill_map.values():
        for skill in skills:
            if skill.lower() in full_text and skill not in matched_skills:
                matched_skills.append(skill)

    return list(set(matched_skills))  # remove duplicates


#function to extract the job level from the title
def extract_job_level(title):
    title = title.lower().strip()

    if any(x in title for x in ["chief", "vp", "vice president", "director", "head", "president", "ceo", "cto", "cfo", "cio", "coo"]):
        return "Executive"
    elif any(x in title for x in ["senior", "lead", "principal", "staff"]):
        return "Senior"
    elif any(x in title for x in ["manager", "mid", "specialist", "consultant"]):
        return "Mid"
    elif any(x in title for x in ["junior", "entry", "associate", "intern"]):
        return "Junior"
    else:
        return "Unspecified"

def estimate_salary(title):
    title = title.lower().strip()

    if "intern" in title:
        return 30000, 50000
    elif any(x in title for x in ["junior", "entry", "associate"]):
        return 60000, 80000
    elif any(x in title for x in ["senior", "lead", "principal", "staff"]):
        return 120000, 160000
    elif any(x in title for x in ["manager", "mid", "specialist", "consultant"]):
        return 100000, 140000
    elif any(x in title for x in ["vp", "vice president", "director", "chief", "head", "president", "ceo", "cto", "cfo", "cio", "coo"]):
        return 250000, 500000
    else:
        return 80000, 110000

# Function to parse salary from a 'benefits' string
def parse_salary_from_benefits(benefits):
    if not isinstance(benefits, str):
        return None, None

    match = re.findall(r"\$([\d.]+)[Kk]?[\/a-z]*\s*-\s*\$([\d.]+)[Kk]?", benefits)
    if match:
        min_val, max_val = match[0]
        min_salary = float(min_val) * (1000 if float(min_val) < 1000 else 1)
        max_salary = float(max_val) * (1000 if float(max_val) < 1000 else 1)
        return int(min_salary), int(max_salary)

    return None, None
#  Simple ASCII-only language filter
def is_english(text):
    try:
        return detect(text) == "en"
    except:
        return False

In [12]:
import requests
import json
import time
from confluent_kafka import Producer
from google.colab import userdata

# Optimized keywords (6 total)
keywords = [
    "data analyst", "machine learning", "software engineer",
    "cloud engineer", "cybersecurity", "product manager", "marketing"
]

seen_jobs = set()
request_count = 0
MAX_REQUESTS = 70
LOCATION_ID = "102571732"  # New York City
PAGE_LIMIT = 2
SLEEP_SECONDS = 6


# Main producer loop
for keyword in keywords:
    if request_count >= MAX_REQUESTS:
        print("Reached API quota.")
        break

    url = "https://linkedin-data-api.p.rapidapi.com/search-jobs-v2"
    params = {
        "keywords": keyword,
        "locationId": LOCATION_ID,
        "datePosted": "anyTime",
        "sort": "mostRelevant"
    }

    try:
        res = requests.get(url, headers=headers, params=params)
        request_count += 1
        print(f"🔍 Requested {keyword}: {res.status_code}")

        if res.status_code != 200:
            print(res.text)
            continue

        jobs = res.json().get("data", [])
        for job in jobs:
            uid = (job["id"], job["title"], job.get("location", ""))
            if uid in seen_jobs:
                continue
            seen_jobs.add(uid)

            title = job["title"]
            if not is_english(title):
                continue
            benefits = job.get("benefits", "")
            description = job.get("description", "")
            salary_min, salary_max = parse_salary_from_benefits(benefits)
            if not salary_min or not salary_max:
                salary_min, salary_max = estimate_salary(title)

            job_record = {
                "job_id": job["id"],
                "title": title,
                "company": job["company"]["name"],
                "location": job.get("location", ""),
                "type": job.get("type", ""),
                "benefits": benefits,
                "description": description,
                "salary_min": salary_min,
                "salary_max": salary_max,
                "job_level": extract_job_level(title),
                "skills_required": extract_skills(title,description),
                "apply_link": job["url"]
            }

            producer.produce("topic_2", key=str(job_record["job_id"]), value=json.dumps(job_record))
            print(f"Produced: {title} @ {job_record['company']}")

    except Exception as e:
        print(f"Error with keyword {keyword}: {e}")

    time.sleep(6)  # avoid hitting quota too fast

producer.flush()
print(f"\n Done streaming! {len(seen_jobs)} unique jobs sent to Kafka.")

🔍 Requested data analyst: 200
Produced: Measurement Intern-Fall 2025 (Unpaid) @ International Rescue Committee
Produced: Data Science Intern - Summer 2025 @ WSP in the U.S.
Produced: Excel Expert (To Start Immediately) @ Vovana (YC W25)
Produced: Tableau Data Analyst Summer Intern @ Authentic Brands Group
Produced: Data Analyst Intern, application via RippleMatch @ RippleMatch
Produced: Data Science Intern– NLP, LLM and GenAI (Early Careers) @ S&P Global
Produced: Data and Applied Scientist @ Microsoft
Produced: SUMMER COLLEGE INTERN - Income @ New York City Campaign Finance Board
Produced: Data Scientist, Finance Analytics - (Texas, New York, or California) @ Jobgether
Produced: Senior Analyst, Analytics Solutions @ Accenture
Produced: Project Specialist (Data Analyst) - Work Management Team - Reporting Group @ Con Edison
Produced: Medical Data Analyst( Professional Coding Only) @ NewYork-Presbyterian Hospital
Produced: Data Analysis Associate @ NFP, an Aon company
Produced: Analyst, 